<p align="center">
  <span style="font-size:22px"><b>ISEL</b></span><br>
  <span style="font-size:22px"><b>Departamento de Engenharia Informática</b></span><br>
  <span style="font-size:22px"><b>Licenciatura em Engenharia Informática e Multimédia</b></span><br>
  <span style="font-size:22px"><b>Processamento de Sinais Multimédia</b></span><br><br>
  <span style="font-size:18px"><b>1º Trabalho Prático</b></span><br>
  <span style="font-size:16px">3º Semestre 2025/2026</span><br><br>
  <span style="font-size:16px">Docente: Prof. Tiago Gonçalves</span><br>
  <span style="font-size:16px">Docente: Prof. Joel Paulo</span><br>
  <span style="font-size:16px">Data: Outobro</span><br><br>
  <span style="font-size:16px">Trabalho realizado por:</span><br>
  <span style="font-size:16px">David Santos nº51417</span><br>
  <span style="font-size:16px">Bernardo Aguiar nº52483</span><br>
  <span style="font-size:16px">Diogo Costa nº52453</span><br>
</p>

##### a) Implemente os Comb-filters na sua estrutura Feadback Comb-Filter $FBCF(α, Dc) = \frac{z^{−Dc}}{1−αz^{−Dc}}$ , onde α está associado ao ganho e Dc ao atraso introduzido (em nº de amostras).

i) Determine e mostre a resposta em frequência (amplitude e fase). Apresente o Diagrama-Polos-Zeros. Nota: Resolva para conjunto de ganhos separadamente.


In [ ]:
## Imports e variáveis globais 

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as ss
import scipy.io.wavfile as wav 

Fs = 44100
N_Pontos = 2048

In [ ]:
# Comb filters
alphas = [0.773, 0.802, 0.753, 0.733]
Dcs = [31.0, 35.0, 39.0, 43.0] 

# All-pass filters
gs = [0.6, 0.5]
Das = [2.6, 1.9] 

In [ ]:
def FBCF(alpha, atraso):
    Dc = int(atraso * (Fs / 1000))

    bks = np.zeros(Dc + 1)
    bks[Dc] = 1.0

    aks = np.zeros(Dc + 1)
    aks[0] = 1.0
    aks[Dc] = -alpha

    w, H = ss.freqz(bks, aks, worN=N_Pontos)
    f = w * Fs / (2 * np.pi)

    zeros = np.roots(bks)
    polos = np.roots(aks)

    return aks, bks, f, H, zeros, polos

for alpha, Dc in zip(alphas, Dcs):
    aks, bks, f,  H, zeros, polos = FBCF(alpha, Dc)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(f, np.abs(H))
    plt.title(f'Resposta em Amplitude do FBCF (alpha={alpha}, Dc={Dc})')
    plt.grid(True)
    plt.xlim(0, 500)
    plt.xlabel('Frequência (Hz)')
    plt.ylabel('Amplitude')

    plt.subplot(1, 2, 2)
    plt.plot(f, np.unwrap(np.angle(H)))
    plt.title('Resposta em Fase do FBCF')
    plt.xlabel('Frequência (Hz)')
    plt.ylabel('Fase (radianos)')
    plt.grid(True)
    plt.xlim(0, Fs/2)
    
    zeros = np.roots(bks)
    polos = np.roots(aks)
    
    # O sistema não possui zeros e possui um número de polos igual ao número de amostras (Dc)
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(np.real(zeros), np.imag(zeros), 'o', markersize=5, color='blue', label='Zeros')    
    plt.title('Diagrama de Zeros')
    plt.xlabel('Eixo Real')
    plt.ylabel('Eixo Imaginário')
    plt.xlim([-1.5, 1.5])
    plt.ylim([-1.5, 1.5])
    plt.grid(True)
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(np.real(polos), np.imag(polos), 'x', markersize=5, color='red', label='Polos')
    plt.title('Diagrama de Polos')
    plt.xlabel('Eixo Real')
    plt.ylabel('Eixo Imaginário')
    plt.xlim([-1.5, 1.5])
    plt.ylim([-1.5, 1.5])
    plt.grid(True)
    plt.legend()

    plt.show()

ii) Determine experimentalmente a resposta impulsional. Ilustre graficamente, guarde num ficheiro wave e escute o resultado.

In [ ]:
atraso = 40
alpha = 0.4
Dc = int(atraso * (Fs / 1000))
DURACAO_SEG = 1     

bks = np.zeros(Dc + 1)
bks[Dc] = 1.0

aks = np.zeros(Dc + 1)
aks[0] = 1.0
aks[Dc] = -alpha

def IR(bks, aks, duration):
    NUM_AMOSTRAS = int(Fs * duration)

    impulso = np.zeros(NUM_AMOSTRAS)
    impulso[0] = 1.0

    h_n = ss.lfilter(bks, aks, impulso)

    return h_n

h_n = IR(bks, aks, DURACAO_SEG)

t = np.arange(0, DURACAO_SEG, 1 / Fs)

plt.figure(figsize=(10, 4))
plt.stem(t * 1000, h_n) 
plt.title(f'Resposta Impulsional do FBCF (alpha={alpha}, Dc={Dc})')
plt.xlabel('Tempo (ms)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.xlim(0, 1000) 
plt.show()

wav.write('fbcf_resposta_impulsional.wav', Fs, h_n.astype(np.float32))

iii) Aplique este filtro a sinais de teste (que considere adequados) e sinais audio para testar a sua funcionalidade.

In [ ]:
bks = np.zeros(Dc + 1)
bks[Dc] = 1.0

aks = np.zeros(Dc + 1)
aks[0] = 1.0
aks[Dc] = -alpha

FREQUENCIA_HZ = 440
AMPLITUDE = 0.2

Dc_list = [1764, 1800, 1850, 1900]
alpha_list = [0.9, 0.85, 0.88, 0.92]

NUM_AMOSTRAS = int(Fs * DURACAO_SEG)
tempo = np.arange(0, DURACAO_SEG, 1/Fs)

sinal_teste = AMPLITUDE * np.cos(2 * np.pi * FREQUENCIA_HZ * tempo) 

pad_length = Fs * 2 
sinal_teste_padded = np.concatenate([sinal_teste, np.zeros(pad_length)])

sinal_processado_total = np.zeros_like(sinal_teste_padded)

for Dc, alpha in zip(Dc_list, alpha_list):
    bks = np.zeros(Dc + 1)
    bks[Dc] = 1.0

    aks = np.zeros(Dc + 1)
    aks[0] = 1.0
    aks[Dc] = -alpha

    sinal_processado_total += ss.lfilter(bks, aks, sinal_teste_padded)

wav.write('fbcf_audio_processado.wav', Fs, sinal_processado_total.astype(np.float32))

#TESTE MAS ESTÁ A FUNCIONAR FIXE

plt.plot(sinal_teste_padded)
plt.show()
plt.plot(sinal_processado_total)

##### b) Implemente os All-Pass filters, com expressão $AP(g,Da) = \frac{-g + z^{-D_a}}{1 - g z^{-D_a}}$ , onde Da representa um atraso (em nº de amostras).

i) Determine e mostre a resposta em frequência (amplitude e fase). Apresente o Diagrama-Polos-Zeros. Nota: Resolva para conjunto de ganhos separadamente

In [ ]:
def APF(g, atraso):
    Da = int(atraso * (Fs / 1000))

    bks = np.zeros(Da + 1)
    bks[0] = -g
    bks[Da] = 1.0

    aks = np.zeros(Da + 1)
    aks[0] = 1.0
    aks[Da] = -g

    w, H = ss.freqz(bks, aks, worN=N_Pontos)
    f = w * Fs / (2 * np.pi)

    zeros = np.roots(bks)
    polos = np.roots(aks)

    return aks, bks, f, H, zeros, polos

aks, bks, f, H, zeros, polos = APF(0.4, 3)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(f, np.abs(H))
plt.title(f'Resposta em Amplitude do AP Filter')
plt.grid(True)
plt.xlim(0, 500)
plt.xlabel('Frequência (Hz)')
plt.ylabel('Amplitude')

plt.subplot(1, 2, 2)
plt.plot(f, np.unwrap(np.angle(H)))
plt.title('Resposta em Fase do AP Filter')
plt.xlabel('Frequência (Hz)')
plt.ylabel('Fase (radianos)')
plt.grid(True)
plt.xlim(0, Fs/2)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(np.real(zeros), np.imag(zeros), 'o', markersize=5, color='blue', label='Zeros')    
plt.title('Diagrama de Zeros')
plt.xlabel('Eixo Real')
plt.ylabel('Eixo Imaginário')
plt.xlim([-1.5, 1.5])
plt.ylim([-1.5, 1.5])
plt.grid(True)
plt.legend()
    
plt.subplot(1, 2, 2)
plt.plot(np.real(polos), np.imag(polos), 'x', markersize=5, color='red', label='Polos')
plt.title('Diagrama de Polos')
plt.xlabel('Eixo Real')
plt.ylabel('Eixo Imaginário')
plt.xlim([-1.5, 1.5])
plt.ylim([-1.5, 1.5])
plt.grid(True)
plt.legend()

plt.show()   

ii) Determine experimentalmente a resposta impulsional. Ilustre graficamente, guarde num ficheiro wave e
escute o resultado.

In [ ]:
aks, bks, f, H, zeros, polos = APF(0.4, 3)

DURACAO_SEG = 1

NUM_AMOSTRAS = int(Fs * DURACAO_SEG)

impulso = np.zeros(NUM_AMOSTRAS)
impulso[0] = 1.0

h_n = ss.lfilter(bks, aks, impulso)

t = np.arange(0, DURACAO_SEG, 1 / Fs)

plt.figure(figsize=(10, 4))
plt.stem(t * 1000, h_n) 
plt.title(f'Resposta Impulsional do AP')
plt.xlabel('Tempo (ms)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.xlim(-5, 150) 
plt.show()

wav.write('ap_resposta_impulsional.wav', Fs, h_n.astype(np.float32))

iii) Aplique este filtro a sinais de teste (que considere adequados) e sinais audio para testar a sua funcionalidade.

In [ ]:
FREQUENCIA_HZ = 440
AMPLITUDE = 0.2

NUM_AMOSTRAS = int(Fs * DURACAO_SEG)
tempo = np.arange(0, DURACAO_SEG, 1/Fs)

sinal_teste = AMPLITUDE * np.cos(2 * np.pi * FREQUENCIA_HZ * tempo) 

pad_length = Fs * 2
sinal_teste_padded = np.concatenate([sinal_teste, np.zeros(pad_length)])

sinal_processado = ss.lfilter(bks, aks, sinal_teste_padded)

wav.write('ap_audio_processado.wav', Fs, sinal_processado.astype(np.float32))

plt.plot(sinal_processado)

#### c) Junte os vários filtros tendo em conta a estrutura do diagrama. Defina parametros α, Dc, g, Da em cada um dos filtros e valide a sua influencia no resultado final.

i) Determine e mostre a resposta em frequência (amplitude e fase).

In [ ]:
H_total_combs = 0
H_total_all_passes = 1
f_Hz = 0

for alpha, Dc in zip(alphas, Dcs):
    _, _, f, H, _, _ = FBCF(alpha, Dc)
    H_total_combs += H
    f_Hz = f

for g, Da in zip(gs, Das):
    _, _, _, H, _, _ = FBCF(alpha, Dc)
    H_total_all_passes *= H

H_total = H_total_combs * H_total_all_passes

# Normalizar
plt.plot(f, np.abs(H_total))
plt.show()
plt.plot(f, np.unwrap(np.angle(H)))

ii) Determine experimentalmente a resposta impulsional. Ilustre graficamente, guarde num ficheiro wave e
escute o resultado.

In [ ]:
h_n_combs = 0
h_n_all_passes = 1

for alpha, Dc in zip(alphas, Dcs):
    aks, bks, _, _, _, _ = FBCF(alpha, Dc)
    h_n = IR(bks, aks, DURACAO_SEG)
    h_n_combs += h_n

for g, Da in zip(gs, Das):
    aks, bks, _, _, _, _ = APF(g, Da)
    h_n = IR(bks, aks, DURACAO_SEG)
    h_n_all_passes = np.convolve(h_n, h_n_all_passes)

# No plot fica tudo zeros
h_n_total = np.convolve(h_n_combs, h_n_all_passes)

plt.stem(h_n_total)

iii) Aplique este filtro a sinais de teste e sinais audio para testar a sua funcionalidade.

In [ ]:
t = np.arange(0, 1, 1/Fs)
sinal_teste = 0.4 * np.cos(2 * np.pi * 440 * t)

pad_length = Fs * 2 
sinal_teste_padded = np.concatenate([sinal_teste, np.zeros(pad_length)])

Fs, x = wav.read("ficheiro.wav")

x = x[:,1]
sinal_processado_total = np.zeros_like(x, dtype=np.float32)

for alpha, Dc in zip(alphas, Dcs):
    aks, bks, _, _, _, _ = FBCF(alpha, Dc)
    sinal_processado_total += ss.lfilter(bks, aks, x)

for g, Da in zip(gs, Das):
    aks, bks, _, _, _, _ = APF(g, Da)
    sinal_processado_total = ss.lfilter(bks, aks, sinal_processado_total)

sinal_processado_total = sinal_processado_total / np.max(np.abs(sinal_processado_total))

wav.write('audio_processado.wav', Fs, sinal_processado_total.astype(np.float32))

plt.plot(sinal_processado_total)